<a href="https://colab.research.google.com/github/gomlfx/ApiaryFund/blob/master/XGBR1_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install flaml

In [36]:
#designed for Jupyter/kaggle/colab
import pandas as pd
import numpy as np
#import matplotlib for plotting 
import matplotlib.pyplot as plt
%matplotlib inline
#import normalization
from sklearn import preprocessing
import xgboost as xg
from sklearn.ensemble import RandomForestRegressor as rf
#import auto hyperparameter tuning
from flaml import AutoML
from xgboost import XGBRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor


In [3]:
 #MT4 csv 
df1=pd.read_csv('EURUSD1440.csv', names=['d','t','eu_o','eu_h','eu_l','eu_c','eu_v'])
df1 = df1.tail(2000)
df2=pd.read_csv('GBPUSD1440.csv', names=['d','t','gu_o','gu_h','gu_l','gu_c','gu_v'])
df2 = df2.tail(2000)

df3=pd.read_csv('USDCAD1440.csv', names=['d','t','uc_o','uc_h','uc_l','uc_c','uc_v'])
df3 = df3.tail(2000)

df4=pd.read_csv('USDCHF1440.csv', names=['d','t','uf_o','uf_h','uf_l','uf_c','uf_v'])
df4 = df4.tail(2000)

df5=pd.read_csv('USDJPY1440.csv', names=['d','t','uj_o','uj_h','uj_l','uj_c','uj_v'])
df5 = df5.tail(2000)

df6=pd.read_csv('USDSEK1440.csv', names=['d','t','us_o','us_h','us_l','us_c','us_v'])
df6 = df6.tail(2000)

#using merge
df_m_1 = df1.merge(df2, on='d')
df_m_2 = df_m_1.merge(df3, on='d')
df_m_3 = df_m_2.merge(df4, on='d')
df_m_4 = df_m_3.merge(df5, on='d')
df_m_5 = df_m_4.merge(df6, on='d')
df_m_5 = df_m_5.drop(columns='t_x')
df_m_5 = df_m_5.drop(columns='t_y')

pd.set_option('display.max_columns', None)
print(df_m_5) 

               d     eu_o     eu_h     eu_l     eu_c    eu_v     gu_o  \
0     2014.03.26  1.38249  1.38263  1.37762  1.37830   65921  1.65249   
1     2014.03.27  1.37834  1.37967  1.37283  1.37437   73853  1.65791   
2     2014.03.28  1.37436  1.37729  1.37047  1.37513   62308  1.66098   
3     2014.03.31  1.37531  1.38085  1.37215  1.37735   71330  1.66550   
4     2014.04.01  1.37734  1.38152  1.37690  1.37932   56207  1.66643   
...          ...      ...      ...      ...      ...     ...      ...   
1993  2021.12.03  1.13005  1.13333  1.12665  1.13088  113218  1.32970   
1994  2021.12.06  1.12997  1.13109  1.12664  1.12848   79067  1.32360   
1995  2021.12.07  1.12821  1.12982  1.12273  1.12681   78783  1.32613   
1996  2021.12.08  1.12650  1.13547  1.12627  1.13418   85355  1.32382   
1997  2021.12.09  1.13422  1.13452  1.13363  1.13371    2695  1.32007   

         gu_h     gu_l     gu_c   gu_v     uc_o     uc_h     uc_l     uc_c  \
0     1.65969  1.65092  1.65807  61534  1.116

In [4]:
df_m_5.tail()

,d,eu_o,eu_h,eu_l,eu_c,eu_v,gu_o,gu_h,gu_l,gu_c,gu_v,uc_o,uc_h,uc_l,uc_c,uc_v,uf_o,uf_h,uf_l,uf_c,uf_v,uj_o,uj_h,uj_l,uj_c,uj_v,us_o,us_h,us_l,us_c,us_v
1993,2021.12.03,1.13005,1.13333,1.12665,1.13088,113218,1.32970,1.33094,1.32081,1.32300,38625,1.28018,1.28536,1.27434,1.28488,38257,0.91998,0.92173,0.91655,0.91786,30287,113.003,113.610,112.556,112.828,57050,9.03786,9.17935,9.03547,9.14200,38831
1994,2021.12.06,1.12997,1.13109,1.12664,1.12848,79067,1.32360,1.32858,1.32220,1.32623,34613,1.28291,1.28422,1.27530,1.27560,32905,0.91716,0.92688,0.91656,0.92551,25926,112.863,113.552,112.846,113.484,51711,9.15248,9.16979,8.98850,9.08968,33157
1995,2021.12.07,1.12821,1.12982,1.12273,1.12681,78783,1.32613,1.32890,1.32085,1.32419,33858,1.27557,1.27669,1.26347,1.26383,30074,0.92478,0.92748,0.92306,0.92486,24666,113.451,113.776,113.397,113.568,40041,9.07722,9.13943,9.04948,9.09611,34249
1996,2021.12.08,1.12650,1.13547,1.12627,1.13418,85355,1.32382,1.32607,1.31605,1.32032,34327,1.26393,1.26659,1.26053,1.26509,32434,0.92480,0.92523,0.91950,0.92046,26375,113.518,113.952,113.307,113.689,43771,9.10286,9.11585,9.01648,9.03649,32737
1997,2021.12.09,1.13422,1.13452,1.13363,1.13371,2695,1.32007,1.32112,1.32007,1.32026,1681,1.26476,1.26607,1.26473,1.26574,1487,0.91935,0.92129,0.91913,0.92083,822,113.639,113.808,113.639,113.795,2264,9.03691,9.04452,9.02916,9.04412,1733


In [5]:
df_m_5.insert(1,'index',df_m_5.index)

In [6]:
#new column: high vs low
df_m_5['eu_h_or_l'] = ((df_m_5.eu_h - df_m_5.eu_o) > (df_m_5.eu_o - df_m_5.eu_l)) 
df_m_5.eu_h_or_l = df_m_5.eu_h_or_l.replace({True:1,False:0})

#new column: close vs yesterday
df_m_5['eu_c_vs_c'] = (df_m_5.eu_c > df_m_5.eu_o) 
df_m_5.eu_c_vs_c = df_m_5.eu_c_vs_c.replace({True:1,False:0})

#new column: shift tomorrow to today
df_m_5['eu_High_next_day'] = df_m_5['eu_h'].shift(-1)
df_m_5['eu_Low_next_day'] = df_m_5['eu_l'].shift(-1)
df_m_5['eu_Close_next_day'] = df_m_5['eu_c'].shift(-1)
df_m_5 = df_m_5.dropna()


In [7]:
df_m_5.tail()

,d,index,eu_o,eu_h,eu_l,eu_c,eu_v,gu_o,gu_h,gu_l,gu_c,gu_v,uc_o,uc_h,uc_l,uc_c,uc_v,uf_o,uf_h,uf_l,uf_c,uf_v,uj_o,uj_h,uj_l,uj_c,uj_v,us_o,us_h,us_l,us_c,us_v,eu_h_or_l,eu_c_vs_c,eu_High_next_day,eu_Low_next_day,eu_Close_next_day
1992,2021.12.02,1992,1.13191,1.13475,1.12952,1.13013,97454,1.32719,1.33331,1.32656,1.32998,37311,1.28133,1.28374,1.27780,1.28048,38280,0.91906,0.92213,0.91780,0.92023,28813,112.725,113.328,112.698,113.185,58505,9.06588,9.09029,9.02448,9.04490,38791,1,0,1.13333,1.12665,1.13088
1993,2021.12.03,1993,1.13005,1.13333,1.12665,1.13088,113218,1.32970,1.33094,1.32081,1.32300,38625,1.28018,1.28536,1.27434,1.28488,38257,0.91998,0.92173,0.91655,0.91786,30287,113.003,113.610,112.556,112.828,57050,9.03786,9.17935,9.03547,9.14200,38831,0,1,1.13109,1.12664,1.12848
1994,2021.12.06,1994,1.12997,1.13109,1.12664,1.12848,79067,1.32360,1.32858,1.32220,1.32623,34613,1.28291,1.28422,1.27530,1.27560,32905,0.91716,0.92688,0.91656,0.92551,25926,112.863,113.552,112.846,113.484,51711,9.15248,9.16979,8.98850,9.08968,33157,0,0,1.12982,1.12273,1.12681
1995,2021.12.07,1995,1.12821,1.12982,1.12273,1.12681,78783,1.32613,1.32890,1.32085,1.32419,33858,1.27557,1.27669,1.26347,1.26383,30074,0.92478,0.92748,0.92306,0.92486,24666,113.451,113.776,113.397,113.568,40041,9.07722,9.13943,9.04948,9.09611,34249,0,0,1.13547,1.12627,1.13418
1996,2021.12.08,1996,1.12650,1.13547,1.12627,1.13418,85355,1.32382,1.32607,1.31605,1.32032,34327,1.26393,1.26659,1.26053,1.26509,32434,0.92480,0.92523,0.91950,0.92046,26375,113.518,113.952,113.307,113.689,43771,9.10286,9.11585,9.01648,9.03649,32737,1,1,1.13452,1.13363,1.13371


In [8]:

df_m_5.isna().sum()

d                    0
index                0
eu_o                 0
eu_h                 0
eu_l                 0
eu_c                 0
eu_v                 0
gu_o                 0
gu_h                 0
gu_l                 0
gu_c                 0
gu_v                 0
uc_o                 0
uc_h                 0
uc_l                 0
uc_c                 0
uc_v                 0
uf_o                 0
uf_h                 0
uf_l                 0
uf_c                 0
uf_v                 0
uj_o                 0
uj_h                 0
uj_l                 0
uj_c                 0
uj_v                 0
us_o                 0
us_h                 0
us_l                 0
us_c                 0
us_v                 0
eu_h_or_l            0
eu_c_vs_c            0
eu_High_next_day     0
eu_Low_next_day      0
eu_Close_next_day    0
dtype: int64

In [9]:
df_m_5.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1997 entries, 0 to 1996
Data columns (total 37 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   d                  1997 non-null   object 
 1   index              1997 non-null   int64  
 2   eu_o               1997 non-null   float64
 3   eu_h               1997 non-null   float64
 4   eu_l               1997 non-null   float64
 5   eu_c               1997 non-null   float64
 6   eu_v               1997 non-null   int64  
 7   gu_o               1997 non-null   float64
 8   gu_h               1997 non-null   float64
 9   gu_l               1997 non-null   float64
 10  gu_c               1997 non-null   float64
 11  gu_v               1997 non-null   int64  
 12  uc_o               1997 non-null   float64
 13  uc_h               1997 non-null   float64
 14  uc_l               1997 non-null   float64
 15  uc_c               1997 non-null   float64
 16  uc_v               1997 

In [10]:
#extract row for variable
last_row = df_m_5.iloc[-1:,np.r_[1:34]].values
print(last_row) 

[[1.99600e+03 1.12650e+00 1.13547e+00 1.12627e+00 1.13418e+00 8.53550e+04
  1.32382e+00 1.32607e+00 1.31605e+00 1.32032e+00 3.43270e+04 1.26393e+00
  1.26659e+00 1.26053e+00 1.26509e+00 3.24340e+04 9.24800e-01 9.25230e-01
  9.19500e-01 9.20460e-01 2.63750e+04 1.13518e+02 1.13952e+02 1.13307e+02
  1.13689e+02 4.37710e+04 9.10286e+00 9.11585e+00 9.01648e+00 9.03649e+00
  3.27370e+04 1.00000e+00 1.00000e+00]]


In [31]:
#for high prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[34]].values
print(y) 

[[0.00000e+00 1.38249e+00 1.38263e+00 ... 5.24030e+04 0.00000e+00
  0.00000e+00]
 [1.00000e+00 1.37834e+00 1.37967e+00 ... 5.89320e+04 0.00000e+00
  0.00000e+00]
 [2.00000e+00 1.37436e+00 1.37729e+00 ... 4.74900e+04 0.00000e+00
  1.00000e+00]
 ...
 [1.99400e+03 1.12997e+00 1.13109e+00 ... 3.31570e+04 0.00000e+00
  0.00000e+00]
 [1.99500e+03 1.12821e+00 1.12982e+00 ... 3.42490e+04 0.00000e+00
  0.00000e+00]
 [1.99600e+03 1.12650e+00 1.13547e+00 ... 3.27370e+04 1.00000e+00
  1.00000e+00]]
[[1.37967]
 [1.37729]
 [1.38085]
 ...
 [1.12982]
 [1.13547]
 [1.13452]]


In [32]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [13]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto")

[flaml.automl: 12-09 01:26:00] {1943} INFO - task = regression
[flaml.automl: 12-09 01:26:00] {1945} INFO - Data split method: uniform
[flaml.automl: 12-09 01:26:00] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-09 01:26:00] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-09 01:26:00] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-09 01:26:00] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-09 01:26:00] {2425} INFO - Estimated sufficient time budget=2359s. Estimated necessary time budget=17s.
[flaml.automl: 12-09 01:26:00] {2505} INFO -  at 0.3s,	estimator lgbm's best error=0.4784,	best estimator lgbm's best error=0.4784
[flaml.automl: 12-09 01:26:00] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-09 01:26:00] {2505} INFO -  at 0.3s,	estimator lgbm's best error=0.4784,	best estimator lgbm's best error=0.4784
[flaml.automl: 12-09 01:26:00] {2311} INFO 

In [33]:
#paste retrained model
sugg_reg_high_param = RandomForestRegressor(max_features=0.7153649632061108, max_leaf_nodes=252,
                      n_estimators=48, n_jobs=-1)
#paste finished
sugg_reg_high_param.fit(X_train,y_train)
y_pred_h=sugg_reg_high_param.predict(last_row)
y_pred_h

NameError: ignored

In [15]:
y_pred = sugg_reg_high_param.predict(X_test)

In [16]:
def mean_absolute_percentage_error(y_true,y_pred):
  y_true, y_pred = np.array(y_true), np.array(y_pred)
  return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

mape = mean_absolute_percentage_error(y_test,y_pred)

In [17]:
#for regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.00299078468335469
MSE: 1.9800257710070122e-05
RMSE: 0.00444974805017881
MAPE: 6.6828670054095625
MedAE 0.0022298754501342


In [18]:
#for low prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[35]].values
print(y) 

[[0.00000e+00 1.38249e+00 1.38263e+00 ... 5.24030e+04 0.00000e+00
  0.00000e+00]
 [1.00000e+00 1.37834e+00 1.37967e+00 ... 5.89320e+04 0.00000e+00
  0.00000e+00]
 [2.00000e+00 1.37436e+00 1.37729e+00 ... 4.74900e+04 0.00000e+00
  1.00000e+00]
 ...
 [1.99400e+03 1.12997e+00 1.13109e+00 ... 3.31570e+04 0.00000e+00
  0.00000e+00]
 [1.99500e+03 1.12821e+00 1.12982e+00 ... 3.42490e+04 0.00000e+00
  0.00000e+00]
 [1.99600e+03 1.12650e+00 1.13547e+00 ... 3.27370e+04 1.00000e+00
  1.00000e+00]]
[[1.37283]
 [1.37047]
 [1.37215]
 ...
 [1.12273]
 [1.12627]
 [1.13363]]


In [19]:
#for low prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [20]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto")

[flaml.automl: 12-09 01:27:02] {1943} INFO - task = regression
[flaml.automl: 12-09 01:27:02] {1945} INFO - Data split method: uniform
[flaml.automl: 12-09 01:27:02] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-09 01:27:02] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-09 01:27:02] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-09 01:27:02] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-09 01:27:02] {2425} INFO - Estimated sufficient time budget=673s. Estimated necessary time budget=5s.
[flaml.automl: 12-09 01:27:02] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4759,	best estimator lgbm's best error=0.4759
[flaml.automl: 12-09 01:27:02] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-09 01:27:03] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4759,	best estimator lgbm's best error=0.4759
[flaml.automl: 12-09 01:27:03] {2311} INFO - 

In [21]:
#paste retrained model
sugg_reg_low_param = ExtraTreesRegressor(max_features=0.4977274222126191, max_leaf_nodes=170,
                    n_estimators=70, n_jobs=-1)
#paste finished
sugg_reg_low_param.fit(X_train,y_train)
y_pred_l=sugg_reg_low_param.predict(last_row)
y_pred_l

array([1.1298257], dtype=float32)

In [22]:
#for low pred 
y_pred = sugg_reg_low_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [23]:
#for low pred regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 

MAE: 0.0028297268513361616
MSE: 1.3353251814740545e-05
RMSE: 0.003654210149230685
MAPE: 6.159919832862507
MedAE 0.0023031997299194362


In [24]:
#for close prediction
#select rows to use for x,y
x = df_m_5.iloc[:,np.r_[1:34]].values
print(x)
y = df_m_5.iloc[:,[36]].values
print(y) 

[[0.00000e+00 1.38249e+00 1.38263e+00 ... 5.24030e+04 0.00000e+00
  0.00000e+00]
 [1.00000e+00 1.37834e+00 1.37967e+00 ... 5.89320e+04 0.00000e+00
  0.00000e+00]
 [2.00000e+00 1.37436e+00 1.37729e+00 ... 4.74900e+04 0.00000e+00
  1.00000e+00]
 ...
 [1.99400e+03 1.12997e+00 1.13109e+00 ... 3.31570e+04 0.00000e+00
  0.00000e+00]
 [1.99500e+03 1.12821e+00 1.12982e+00 ... 3.42490e+04 0.00000e+00
  0.00000e+00]
 [1.99600e+03 1.12650e+00 1.13547e+00 ... 3.27370e+04 1.00000e+00
  1.00000e+00]]
[[1.37437]
 [1.37513]
 [1.37735]
 ...
 [1.12681]
 [1.13418]
 [1.13371]]


In [25]:
#for close prediction
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size = 0.30)

In [26]:
automl_reg = AutoML()
automl_reg.fit(X_train, y_train, estimator_list="auto", task="regression", time_budget=None, eval_method="auto")

[flaml.automl: 12-09 01:28:05] {1943} INFO - task = regression
[flaml.automl: 12-09 01:28:05] {1945} INFO - Data split method: uniform
[flaml.automl: 12-09 01:28:05] {1949} INFO - Evaluation method: cv
[flaml.automl: 12-09 01:28:05] {2019} INFO - Minimizing error metric: 1-r2
[flaml.automl: 12-09 01:28:05] {2071} INFO - List of ML learners in AutoML Run: ['lgbm', 'rf', 'xgboost', 'extra_tree', 'xgb_limitdepth']
[flaml.automl: 12-09 01:28:05] {2311} INFO - iteration 0, current learner lgbm
[flaml.automl: 12-09 01:28:05] {2425} INFO - Estimated sufficient time budget=596s. Estimated necessary time budget=4s.
[flaml.automl: 12-09 01:28:05] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4832,	best estimator lgbm's best error=0.4832
[flaml.automl: 12-09 01:28:05] {2311} INFO - iteration 1, current learner lgbm
[flaml.automl: 12-09 01:28:05] {2505} INFO -  at 0.1s,	estimator lgbm's best error=0.4832,	best estimator lgbm's best error=0.4832
[flaml.automl: 12-09 01:28:05] {2311} INFO - 

In [27]:
#paste retrained model
sugg_reg_close_param = RandomForestRegressor(max_features=0.5946562283064879, max_leaf_nodes=43,
                      n_estimators=32, n_jobs=-1)
#paste finished
sugg_reg_close_param.fit(X_train,y_train)
y_pred_c=sugg_reg_close_param.predict(last_row)
y_pred_c

array([1.1345048], dtype=float32)

In [28]:
#for close prediction
y_pred = sugg_reg_close_param.predict(X_test)
mape = mean_absolute_percentage_error(y_test,y_pred)

In [29]:
#for close regression score
from sklearn import metrics 
print('MAE:', metrics.mean_absolute_error(y_test,y_pred))
print('MSE:', metrics.mean_squared_error(y_test, y_pred))
print('RMSE:', np.sqrt(metrics.mean_squared_error(y_test, y_pred)))
print('MAPE:', mape)
print('MedAE', metrics.median_absolute_error(y_test,y_pred)) 
#print('MdAPE', metrics)

MAE: 0.004291248540814718
MSE: 3.175752109348742e-05
RMSE: 0.0056353811843998115
MAPE: 6.158433452384354
MedAE 0.003492588901519711


In [30]:
print('Today the predictions are:')
print('high', y_pred_h)
print('close', y_pred_c)
print('low', y_pred_l)
print('Best used as polarities.')

Today the predictions are:
high [1.1382235]
close [1.1345048]
low [1.1298257]
Best used as polarities.
